# E-Mail Reminder for tracking attendance in Konzertmeister

In [0]:
query_aptmts = open('appointments_no_attendance.sql', 'r').read()
aptmts_df = spark.sql(query_aptmts) 
display(aptmts_df)

In [0]:
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


def send_mails_aptms(df_aptmt):
    """
    Send reminder emails for appointments with missing attendance.

    Parameters
    ----------
    df_aptmt : pyspark.sql.DataFrame
        DataFrame containing appointment details with missing attendance.

    Returns
    -------
    None
    """
    # Set up the SMTP server
    smtp_server = "smtp.office365.com"
    smtp_port = 587
    smtp_user = dbutils.secrets.get(scope="bmk-key-vault-scope", key="sp-news-bmk-mail")
    smtp_password = dbutils.secrets.get(
        scope="bmk-key-vault-scope", key="sp-news-bmk-pw"
    )

    # Create the message
    msg = MIMEMultipart("alternative")
    msg["Subject"] = f"Fehlende Anwesenheiten in Konzertmeister - {df_aptmt.first()['appointment_name']}"
    msg["From"] = "BMK Service User <" + smtp_user + ">"
    msg["Cc"] = "manuel.guth@bmk-buggingen.de"

    # define info for Mail
    att_url = f"https://web.konzertmeister.app/appointment/{df_aptmt.first()['appointmentId']}/attendance"
    # check which leaders should receive mail
    if df_aptmt.count() == 1:
        msg["To"] = df_aptmt.first()["mail"]
        intro = f"Hallo {df_aptmt.first()['name']},"
    else:
        intro = "Hallo zusammen, \n\n"
        df_aptmt = df_aptmt.filter(df_aptmt.Register != "Dirigent")
        msg["To"] = ", ".join([row["mail"] for row in df_aptmt.collect()])
    body = f"""

die Veranstaltung `{df_aptmt.first()['appointment_name']}` fand am {df_aptmt.first()['appointment_date'].strftime('%d.%m.%Y')} statt.

Bisher wurden noch keine Anwesenheiten eingetragen, diese können unter {att_url} eingetragen werden.
Bitte tragt ein, wer bei dieser Veranstaltung anwesend war.

Euer BMK Bot :)
    """
    text_content = intro + body
    # Attach the plain text and HTML content to the message
    text_part = MIMEText(text_content, "plain")
    msg.attach(text_part)
    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_user, smtp_password)
    server.sendmail(smtp_user, msg["To"].split(", ") + msg["Cc"].split(", "), msg.as_string())
    server.quit()

In [0]:
appointment_ids = list(set([row['appointmentId'] for row in aptmts_df.collect()]))
for appointment_id in appointment_ids:
    df_aptmt_i = aptmts_df.filter(aptmts_df.appointmentId == appointment_id)
    send_mails_aptms(df_aptmt_i)